# Modelo de Identificação de Fraudes

- Informações da base:
    - 'Time' (tempo): contém os segundos decorridos entre cada transação e a primeira transação no conjunto de dados. 
    - 'Amount' (valor): é o valor da transação 
    - 'Class' (classe): é a variável de resposta e assume valor 1 em caso de fraude e 0 caso contrário.
    - "Infelizmente, devido a questões de confidencialidade, não podemos fornecer os recursos originais e mais informações básicas sobre os dados. Características V1, V2, … V28 são os principais componentes obtidos com PCA"

In [1]:
# Configuração do ambiente

# Garante que o notebook está na raiz do projeto
%cd .. 

# Verifica o diretório atual (Linux/Mac)
# !pwd  

# Verifica o diretório atual (Windows)
!cd 

C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude
C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude


In [2]:
# Importações necessárias 

import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt
import xgboost as xgb

from scipy.stats import kstest, normaltest, anderson

from sklearn.preprocessing import PowerTransformer, MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids, NearMiss 
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.combine import SMOTEENN

from sklearn.metrics import (
    confusion_matrix, 
    accuracy_score, 
    precision_score, 
    recall_score,
    precision_recall_curve,
)

import warnings
from src.config.paths import DADOS_CREDICARD, DADOS_CREDICARD_TRATADO
from src.config.auxiliares_ml import downcast_dataframe, testar_modelos_com_undersampling

warnings.filterwarnings('ignore')
sns.set_theme(palette="bright")

In [3]:
# Importando a base

transacoes = pd.read_parquet(DADOS_CREDICARD_TRATADO)

In [4]:
# Separando X e y

X = transacoes.drop(columns='Class')
y = transacoes['Class']

In [5]:
# Separando em treino e teste

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0,stratify=y)

### OBSERVAÇÃO:
**Podemos observar que, já que queremos detectar fraudes, ou seja o que é 1, precisamos melhorar a precisão e o recall.**

**PRÉ-PROCESSAMENTO**: Fazendo o balanceamento com UnderSampler

### Utilizando o RandomUnderSampler

In [6]:
# Definindo o RandomUnderSampler

rus = RandomUnderSampler(random_state=42)

# Definindo a nova amostra

X_resRUS, y_resRUS = rus.fit_resample(X_train, y_train)

### XGBoost 

In [7]:
# Criando o modelo XGBoost

modelo_xgb_RUS = xgb.XGBClassifier(
    max_depth=9,
    n_estimators=200,
    learning_rate=0.3,
    random_state=0,
    scale_pos_weight=(y.value_counts()[0] / y.value_counts()[1])  # Lida com desbalanceamento
)

# Treinando e avaliando o modelo

modelo_xgb_RUS.fit(X_resRUS, y_resRUS)
y_pred_xgb_RUS = modelo_xgb_RUS.predict(X_test)

# Calculando as métricas

acc_xgb = accuracy_score(y_test, y_pred_xgb_RUS)
prec_xgb = precision_score(y_test, y_pred_xgb_RUS)
rec_xgb = recall_score(y_test, y_pred_xgb_RUS)

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_pred_xgb_RUS)

array([[87263,  6562],
       [   14,   148]])

In [8]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_pred_xgb_RUS))
print('Precisão:', precision_score(y_test, y_pred_xgb_RUS))
print('Recall:', recall_score(y_test, y_pred_xgb_RUS, pos_label=1))

Acurácia: 0.9300328768872291
Precisão: 0.022056631892697468
Recall: 0.9135802469135802


### Random Forest

In [9]:
# Utilizando o Random Forest

clf_RF_RUS = RandomForestClassifier(
    max_depth=7,  
    random_state=0,
    criterion='gini',
    n_estimators=100,  
    class_weight='balanced_subsample' 
).fit(X_resRUS, y_resRUS)

y_pred_RF_RUS = clf_RF_RUS.predict(X_test)

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_pred_RF_RUS)

array([[92022,  1803],
       [   21,   141]])

In [10]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_pred_RF_RUS))
print('Precisão:', precision_score(y_test, y_pred_RF_RUS))
print('Recall:', recall_score(y_test, y_pred_RF_RUS))

Acurácia: 0.9805930607424431
Precisão: 0.07253086419753087
Recall: 0.8703703703703703


### Regressão Logística

In [11]:
# Utilizando a Regressão Logística

clfLog_RUS = LogisticRegression(random_state=0).fit(X_resRUS, y_resRUS)

# Calculando as métricas

y_predLog_RUS = clfLog_RUS.predict(X_test)
y_pred_probaLog_RUS = clfLog_RUS.predict_proba(X_test)[:,1]

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_predLog_RUS)

array([[90513,  3312],
       [   16,   146]])

In [12]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_predLog_RUS))
print('Precisão:', precision_score(y_test, y_predLog_RUS))
print('Recall:', recall_score(y_test, y_predLog_RUS))

Acurácia: 0.9645908476704225
Precisão: 0.042220936957779065
Recall: 0.9012345679012346


### SVC

In [13]:
# Utilizando o SVC

clfSVC_RUS = SVC(random_state=0, probability=True).fit(X_resRUS, y_resRUS)

# Fazendo a previsão

y_predSVC_RUS = clfSVC_RUS.predict(X_test)
y_pred_probaSVC_RUS = clfSVC_RUS.predict_proba(X_test)[:,1]

# Visualizando a matriz confusão

confusion_matrix(y_test, y_predSVC_RUS)

array([[42859, 50966],
       [   54,   108]])

In [14]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_predSVC_RUS))
print('Precisão:', precision_score(y_test, y_predSVC_RUS))
print('Recall:', recall_score(y_test, y_predSVC_RUS))

Acurácia: 0.45715896879355655
Precisão: 0.0021145788463797625
Recall: 0.6666666666666666


### Utilizando o ClusterCentroids

In [15]:
# Definindo ClusterCentroids

cc = ClusterCentroids(random_state=42)

# Criando a amostra dos dados

X_res_CC, y_res_CC = cc.fit_resample(X_train, y_train)

### XGBoost 

In [16]:
# Criando o modelo XGBoost

modelo_xgb_CC = xgb.XGBClassifier(
    max_depth=9,
    n_estimators=200,
    learning_rate=0.3,
    random_state=0,
    scale_pos_weight=(y.value_counts()[0] / y.value_counts()[1])  # Lida com desbalanceamento
)

# Treinando e avaliando o modelo

modelo_xgb_CC.fit(X_res_CC, y_res_CC)
y_pred_xgb_CC = modelo_xgb_CC.predict(X_test)

# Calculando as métricas

acc_xgb = accuracy_score(y_test, y_pred_xgb_CC)
prec_xgb = precision_score(y_test, y_pred_xgb_CC)
rec_xgb = recall_score(y_test, y_pred_xgb_CC)

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_pred_xgb_CC)

array([[16738, 77087],
       [    0,   162]])

In [17]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_pred_xgb_CC))
print('Precisão:', precision_score(y_test, y_pred_xgb_CC))
print('Recall:', recall_score(y_test, y_pred_xgb_CC, pos_label=1))

Acurácia: 0.1798121016736357
Precisão: 0.0020971145257543786
Recall: 1.0


### Random Forest

In [18]:
# Utilizando o Random Forest

clf_RF_CC = RandomForestClassifier(
    max_depth=7,  
    random_state=0,
    criterion='gini',
    n_estimators=100,  
    class_weight='balanced_subsample' 
).fit(X_res_CC, y_res_CC)

y_pred_RF_CC = clf_RF_CC.predict(X_test)

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_pred_RF_CC)

array([[ 7252, 86573],
       [    1,   161]])

In [19]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_pred_RF_CC))
print('Precisão:', precision_score(y_test, y_pred_RF_CC))
print('Recall:', recall_score(y_test,y_pred_RF_CC))

Acurácia: 0.0788726100418143
Precisão: 0.0018562501441188
Recall: 0.9938271604938271


### Regressão Logística

In [20]:
# Utilizando a Regressão Logística

clfLog_CC = LogisticRegression(random_state=0).fit(X_res_CC, y_res_CC)

# Calculando as métricas

y_predLog_CC = clfLog_CC.predict(X_test)
y_pred_probaLog_CC = clfLog_RUS.predict_proba(X_test)[:,1]

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_predLog_CC)

array([[84465,  9360],
       [   12,   150]])

In [21]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_predLog_CC))
print('Precisão:', precision_score(y_test, y_predLog_CC))
print('Recall:', recall_score(y_test, y_predLog_CC))

Acurácia: 0.9002840818411056
Precisão: 0.015772870662460567
Recall: 0.9259259259259259


### SVC

In [22]:
# Utilizando o SVC

clfSVC_CC = SVC(random_state=0, probability=True).fit(X_res_CC, y_res_CC)

# Fazendo a previsão

y_predSVC_CC = clfSVC_CC.predict(X_test)
y_pred_probaSVC_CC = clfSVC_CC.predict_proba(X_test)[:,1]

# Visualizando a matriz confusão

confusion_matrix(y_test, y_predSVC_CC)

array([[44889, 48936],
       [   79,    83]])

In [23]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_predSVC_CC))
print('Precisão:', precision_score(y_test, y_predSVC_CC))
print('Recall:', recall_score(y_test, y_predSVC_CC))

Acurácia: 0.4784917062998074
Precisão: 0.0016932209959403496
Recall: 0.5123456790123457


### Utilizando o NearMiss

In [24]:
# Utilizando o NearMiss

nm = NearMiss()

# Criando a amostra dos dados

X_res_NM, y_res_NM = nm.fit_resample(X_train, y_train)

### XGBoost 

In [25]:
# Criando o modelo XGBoost

modelo_xgb_NM = xgb.XGBClassifier(
    max_depth=9,
    n_estimators=200,
    learning_rate=0.3,
    random_state=0,
    scale_pos_weight=(y.value_counts()[0] / y.value_counts()[1])  # Lida com desbalanceamento
)

# Treinando e avaliando o modelo

modelo_xgb_NM.fit(X_res_NM, y_res_NM)
y_pred_xgb_NM = modelo_xgb_NM.predict(X_test)

# Calculando as métricas

acc_xgb = accuracy_score(y_test, y_pred_xgb_NM)
prec_xgb = precision_score(y_test, y_pred_xgb_NM)
rec_xgb = recall_score(y_test, y_pred_xgb_NM)

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_pred_xgb_NM)

array([[66431, 27394],
       [    7,   155]])

In [26]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_pred_xgb_NM))
print('Precisão:', precision_score(y_test, y_pred_xgb_NM))
print('Recall:', recall_score(y_test, y_pred_xgb_NM, pos_label=1))

Acurácia: 0.7084596805941247
Precisão: 0.005626338524084359
Recall: 0.9567901234567902


### Random Forest

In [27]:
# Utilizando o Random Forest

clf_RF_NM = RandomForestClassifier(
    max_depth=7,  
    random_state=0,
    criterion='gini',
    n_estimators=100,  
    class_weight='balanced_subsample' 
).fit(X_res_NM, y_res_NM)

y_pred_RF_NM = clf_RF_NM.predict(X_test)

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_pred_RF_NM)

array([[71471, 22354],
       [   13,   149]])

In [28]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_pred_RF_NM))
print('Precisão:', precision_score(y_test, y_pred_RF_NM))
print('Recall:', recall_score(y_test,y_pred_RF_NM))

Acurácia: 0.7620202794003426
Precisão: 0.006621339376971959
Recall: 0.9197530864197531


### SVC

In [29]:
# Utilizando o SVC

clfSVC_NM = SVC(random_state=0, probability=True).fit(X_res_NM, y_res_NM)

# Fazendo a previsão

y_predSVC_NM = clfSVC_NM.predict(X_test)
y_pred_probaSVC_NM = clfSVC_NM.predict_proba(X_test)[:,1]

# Visualizando a matriz confusão

confusion_matrix(y_test, y_predSVC_NM)

array([[39470, 54355],
       [   52,   110]])

In [30]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_predSVC_NM))
print('Precisão:', precision_score(y_test, y_predSVC_NM))
print('Recall:', recall_score(y_test, y_predSVC_NM))

Acurácia: 0.4211220700735208
Precisão: 0.002019645643991554
Recall: 0.6790123456790124


### OBSERVAÇÕES: 

- Em geral os resultados não foram bons. <br> 
- O único balanceamento que faz algum sentido é com RandomUnderSampler.<br>
- Porém, há um desequilíbrio muito grande entre precisão e recall.<br>
- Isso vai contra as expectativas da empresa para o modelo em questão.